In [1]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!python -m pip install nemo_toolkit[all]
exit()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Using cached matplotlib-3.5.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)


In [1]:
data_dir = '/content'

In [5]:
! ls /content

drive		QuartzNet15x5_golos.nemo  test	    train
lightning_logs	sample_data		  test.tar  train_farfield.tar


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import glob
import os
import subprocess
import tarfile
import wget

# Загрузка датасета Golos
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

In [4]:

test_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(test_path)
    tar.extractall(path=data_dir)

train_path = load("https://sc.link/1Z3", "train_farfield.tar")

if not os.path.exists(os.path.join(data_dir, '/train/')):
    tar = tarfile.open(train_path)
    tar.extractall(path=data_dir)


Dataset downloaded at: /content/test.tar
Dataset downloaded at: /content/train_farfield.tar


In [ ]:
import librosa
import IPython.display as ipd

# Load and listen to the audio file
example_file = data_dir + '/train/farfield/000f6cd3b67ec7eea4fa122dbc0b4852.wav'

audio, sample_rate = librosa.load(example_file)

ipd.Audio(example_file, rate=sample_rate)

In [3]:
import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-30 15:57:09 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [11]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

Tarfile already exists.


[NeMo W 2021-11-30 15:58:19 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-30 15:58:19 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-30 15:58:19 features:265] PADDING: 16
[NeMo I 2021-11-30 15:58:19 features:282] STFT using torch
[NeMo I 2021-11-30 15:58:24 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


In [13]:
import json
import random 
# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs, farfield=False):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
          if farfield:
            counter=0
            for line in fin:
                chance = random.random()
                if chance>0.3:
                  continue
                if counter > 40000:
                    break
                metadata = json.loads(line)
                if metadata["duration"]>0.1:
                  metadata["audio_filepath"]=os.path.join("/content/train/farfield", metadata["audio_filepath"])
                  json.dump(metadata, fout)
                  fout.write('\n')
                  counter+=1
            print(counter)
          else:
            for line in fin:
                metadata = json.loads(line)
                metadata["audio_filepath"]=os.path.join("/content/test/farfield/files", metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')
                
# Building Manifests
print("******")
test_rel = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
test_abs = os.path.join(data_dir, 'test/farfield/farfield_test.jsonl')
if not os.path.isfile(test_abs):
  build_manifest(test_rel, test_abs)
test_manifest =test_abs
print("test_manifest", test_manifest)

******
test_manifest /content/test/farfield/farfield_test.jsonl


In [14]:
! head /content/test/farfield/farfield_test.jsonl


{"id": "58b586f67f5e634506e215df5996b82e", "audio_filepath": "/content/test/farfield/files/58b586f67f5e634506e215df5996b82e.wav", "text": "\u0434\u0436\u043e\u0439 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.696625}
{"id": "26093ef7a8c5ec6a3c586a6a929c1bd3", "audio_filepath": "/content/test/farfield/files/26093ef7a8c5ec6a3c586a6a929c1bd3.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0432\u044b\u0437\u043e\u0432 \u0441\u0432\u0435\u0442\u043b\u0430\u043d\u0435 \u0432\u0430\u0441\u0438\u043b\u044c\u0435\u0432\u043d\u0435 \u043d\u0438\u043a\u043e\u043b\u0435\u043d\u043a\u043e", "duration": 3.1720625}
{"id": "b42d13ab3dc159fac39a1120e8011f66", "audio_filepath": "/content/test/farfield/files/b42d13ab3dc159fac39a1120e8011f66.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.5226875}
{"id": "7e55843e745a239b35212e2caba77239", "audio_filepath": "/content/test/farfield/files/7e55843e745a239b35212e2caba77239.wav", "text": "\u0434\u0436\

In [15]:
print("******")
train_rel = "/content/drive/MyDrive/data/train_farfield.jsonl"
train_abs = os.path.join(data_dir, 'train/farfield/farfield_train_n.jsonl')

# if not os.path.isfile(train_abs):
build_manifest(train_rel, train_abs, farfield=True)
train_manifest = train_abs
print("train_manifest", train_manifest)

******
37297
train_manifest /content/train/farfield/farfield_train_n.jsonl


In [16]:
! head /content/train/farfield/farfield_train_n.jsonl

{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "/content/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0445\u043e\u0442\u0435\u043b\u0430 \u043d\u043e\u043c\u0435\u0440 \u0442\u0435\u043b\u0435\u0444\u043e\u043d\u0430 \u043f\u043e\u043c\u0435\u043d\u044f\u0442\u044c \u043a\u0430\u0440\u0442\u044b", "duration": 5.4596875}
{"id": "566e4f7d788d755e2fa76ab57837d7dd", "audio_filepath": "/content/train/farfield/566e4f7d788d755e2fa76ab57837d7dd.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u043f\u043b\u043e\u0449\u0430\u0434\u044c \u043a\u043e\u0440\u043e\u043b\u0435\u0432\u0441\u0442\u0432\u0430 \u0442\u043e\u043d\u0433\u0430", "duration": 4.75025}
{"id": "a46c2afa0ca8a678644677db869044d9", "audio_filepath": "/content/train/farfield/a46c2afa0ca8a678644677db869044d9.wav", "text": "\u0434\u0436\u043e\u0439 \u0435\u0441\u043b\u0438 \u044f \u043f\u043e\u0434\u0442\u0432\u0435\u0440\u0436\u0434\u0430\u044e \u043e\u043f\u0435\u0440\u

In [8]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=1, precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [17]:
params_model = asr_model._cfg

# --- Config Information ---#
# try:
#     from ruamel.yaml import YAML
# except ModuleNotFoundError:
#     from ruamel_yaml import YAML
# config_path = '/usr/local/lib/python3.7/dist-packages/nemo/collections/asr/models/configs/'

# yaml = YAML(typ='safe')
# with open(config_path) as f:
#     params = yaml.load(f)
# print(params)
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16

In [18]:
params_model['optim']['lr'] = 0.0005
params_model

{'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'optim': {'name': 'novograd', 'lr': 0.0005, 'betas': [0.9, 0.98], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': 500, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}, 'train_ds': {'manifest_filepath': '/content/train/farfield/farfield_train_n.jsonl', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'num_workers': 20, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'tarred_shard_strategy': 'scatter', 'parser': 'base'}, 'validation_ds': {'manifest_filepath':

In [24]:
from omegaconf import DictConfig
import copy
first_asr_model = asr_model
new_opt = copy.deepcopy(params_model['optim'])
new_opt['lr'] = 0.00015
first_asr_model.setup_optimization(optim_config=DictConfig(new_opt))

[NeMo W 2021-11-30 16:00:40 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 16:00:40 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.00015
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 16:00:40 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     eps: 1e-08
     grad_averaging: False
     lr: 0.00015
     weight_decay: 0.001
 ), None)

In [25]:
first_asr_model.setup_training_data(train_data_config=params_model['train_ds'])
first_asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])
first_asr_model.setup_test_data(test_data_config=params_model['test_ds'])

[NeMo I 2021-11-30 16:00:45 collections:173] Dataset loaded with 37297 files totalling 39.89 hours
[NeMo I 2021-11-30 16:00:45 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-30 16:00:45 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 16:00:45 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 16:00:45 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 16:00:45 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 16:00:45 collections:174] 0 files were filtered totalling 0.00 hours


In [26]:
first_asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())
        
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 16:00:46 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.6560486757337151


In [27]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=2, precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [28]:
trainer.fit(first_asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 16:03:49 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 16:03:49 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.00015
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 16:03:49 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 16:03:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-30 16:03:52 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [29]:
first_asr_model.save_to('/content/drive/MyDrive/data/model_save.nemo')

In [77]:
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16
params_model['optim']['lr'] = 0.00015

In [78]:
first_asr_model.setup_training_data(train_data_config=params_model['train_ds'])
first_asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])
first_asr_model.setup_test_data(test_data_config=params_model['test_ds'])

[NeMo I 2021-11-30 18:50:36 collections:173] Dataset loaded with 37297 files totalling 39.89 hours
[NeMo I 2021-11-30 18:50:36 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-30 18:50:36 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 18:50:36 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 18:50:36 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 18:50:36 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 18:50:36 collections:174] 0 files were filtered totalling 0.00 hours


In [79]:
first_asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.

for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())
        
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 18:50:37 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.5384156525888809
